# Expedia 호텔 예약 예측 
- kaggle, https://www.kaggle.com/c/expedia-hotel-recommendations

In [1]:
sc

In [3]:
import collections
import time

from pyspark import StorageLevel
from pyspark.sql import Row
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import functions
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [4]:
spark = SparkSession \
    .builder \
    .appName("project") \
    .master("local[*]") \
    .getOrCreate()

In [13]:
train_data = spark.read.csv("file:///home/ubuntu/Session/180602_project/data/train.csv")
test_data = spark.read.csv("file:///home/ubuntu/Session/180602_project/data/test.csv")
destination_data = spark.read.csv("file:///home/ubuntu/Session/180602_project/data/destinations.csv")

## 전처리

### rename
- column이름 바꾸기

In [4]:
col_name = train_data.columns
print(col_name)

['_c0', '_c1', '_c2', '_c3', '_c4', '_c5', '_c6', '_c7', '_c8', '_c9', '_c10', '_c11', '_c12', '_c13', '_c14', '_c15', '_c16', '_c17', '_c18', '_c19', '_c20', '_c21', '_c22', '_c23']


In [5]:
real_name = train_data.take(1)
real_name

[Row(_c0='date_time', _c1='site_name', _c2='posa_continent', _c3='user_location_country', _c4='user_location_region', _c5='user_location_city', _c6='orig_destination_distance', _c7='user_id', _c8='is_mobile', _c9='is_package', _c10='channel', _c11='srch_ci', _c12='srch_co', _c13='srch_adults_cnt', _c14='srch_children_cnt', _c15='srch_rm_cnt', _c16='srch_destination_id', _c17='srch_destination_type_id', _c18='is_booking', _c19='cnt', _c20='hotel_continent', _c21='hotel_country', _c22='hotel_market', _c23='hotel_cluster')]

In [6]:
new_col_name = []
for idx in range(24) :
    new_col_name.append(train_data.take(1)[0][idx])
    
print(new_col_name)

['date_time', 'site_name', 'posa_continent', 'user_location_country', 'user_location_region', 'user_location_city', 'orig_destination_distance', 'user_id', 'is_mobile', 'is_package', 'channel', 'srch_ci', 'srch_co', 'srch_adults_cnt', 'srch_children_cnt', 'srch_rm_cnt', 'srch_destination_id', 'srch_destination_type_id', 'is_booking', 'cnt', 'hotel_continent', 'hotel_country', 'hotel_market', 'hotel_cluster']


In [67]:
command = []
for idx in range(24) :
    command.append(col(col_name[idx]).alias(new_col_name[idx]))
    
train_data = train_data.select(command)

In [68]:
train_data = train_data.filter(train_data.date_time != 'date_time')
train_data.show(3)

+-------------------+---------+--------------+---------------------+--------------------+------------------+-------------------------+-------+---------+----------+-------+----------+----------+---------------+-----------------+-----------+-------------------+------------------------+----------+---+---------------+-------------+------------+-------------+
|          date_time|site_name|posa_continent|user_location_country|user_location_region|user_location_city|orig_destination_distance|user_id|is_mobile|is_package|channel|   srch_ci|   srch_co|srch_adults_cnt|srch_children_cnt|srch_rm_cnt|srch_destination_id|srch_destination_type_id|is_booking|cnt|hotel_continent|hotel_country|hotel_market|hotel_cluster|
+-------------------+---------+--------------+---------------------+--------------------+------------------+-------------------------+-------+---------+----------+-------+----------+----------+---------------+-----------------+-----------+-------------------+------------------------+--

In [14]:
## 함수 정의
def making_dataframe(df) :
    col_name = df.columns
    real_name = df.take(1)
    
    new_col_name = []
    for idx in range(len(col_name)) :
        new_col_name.append(real_name[0][idx])
        
    command = []
    for idx in range(len(col_name)) :
        command.append(col(col_name[idx]).alias(new_col_name[idx]))
        
    df = df.select(command)
    df = df.filter(df[real_name[0][0]] != real_name[0][0])
    
    return df

In [15]:
train_df = making_dataframe(train_data)
test_df = making_dataframe(test_data)

In [6]:
train_df.show(3)
print("\n\n\n")
test_df.show(3)

+-------------------+---------+--------------+---------------------+--------------------+------------------+-------------------------+-------+---------+----------+-------+----------+----------+---------------+-----------------+-----------+-------------------+------------------------+----------+---+---------------+-------------+------------+-------------+
|          date_time|site_name|posa_continent|user_location_country|user_location_region|user_location_city|orig_destination_distance|user_id|is_mobile|is_package|channel|   srch_ci|   srch_co|srch_adults_cnt|srch_children_cnt|srch_rm_cnt|srch_destination_id|srch_destination_type_id|is_booking|cnt|hotel_continent|hotel_country|hotel_market|hotel_cluster|
+-------------------+---------+--------------+---------------------+--------------------+------------------+-------------------------+-------+---------+----------+-------+----------+----------+---------------+-----------------+-----------+-------------------+------------------------+--

### 코드가 실행되는지 확인하기 위한 data 추출

In [16]:
train_temp = train_df.take(100)
test_temp = test_df.take(100)

train_df = spark.createDataFrame(train_temp)
test_df = spark.createDataFrame(test_temp)

In [8]:
train_df.show()
print("\n\n\n")
test_df.show()

+-------------------+---------+--------------+---------------------+--------------------+------------------+-------------------------+-------+---------+----------+-------+----------+----------+---------------+-----------------+-----------+-------------------+------------------------+----------+---+---------------+-------------+------------+-------------+
|          date_time|site_name|posa_continent|user_location_country|user_location_region|user_location_city|orig_destination_distance|user_id|is_mobile|is_package|channel|   srch_ci|   srch_co|srch_adults_cnt|srch_children_cnt|srch_rm_cnt|srch_destination_id|srch_destination_type_id|is_booking|cnt|hotel_continent|hotel_country|hotel_market|hotel_cluster|
+-------------------+---------+--------------+---------------------+--------------------+------------------+-------------------------+-------+---------+----------+-------+----------+----------+---------------+-----------------+-----------+-------------------+------------------------+--

### is_booking 삭제

In [17]:
def is_booking(df) :
    new_df = df.where(df.is_booking == 1)
    new_df = new_df.drop("is_booking").select("*")
    
    return new_df

In [18]:
train_df = is_booking(train_df)
train_df.show()

+-------------------+---------+--------------+---------------------+--------------------+------------------+-------------------------+-------+---------+----------+-------+----------+----------+---------------+-----------------+-----------+-------------------+------------------------+---+---------------+-------------+------------+-------------+
|          date_time|site_name|posa_continent|user_location_country|user_location_region|user_location_city|orig_destination_distance|user_id|is_mobile|is_package|channel|   srch_ci|   srch_co|srch_adults_cnt|srch_children_cnt|srch_rm_cnt|srch_destination_id|srch_destination_type_id|cnt|hotel_continent|hotel_country|hotel_market|hotel_cluster|
+-------------------+---------+--------------+---------------------+--------------------+------------------+-------------------------+-------+---------+----------+-------+----------+----------+---------------+-----------------+-----------+-------------------+------------------------+---+---------------+----

### NA 처리1

In [19]:
def drop_na(df) :
    new_df = df.na.drop()
    
    return new_df

In [20]:
def fill_na(df, ctg) :
    if ctg == "train" :
        new_df = df.na.fill({"date_time":"2013.5-08-00 00:00:00", "srch_ci":"2013.5-09-00","srch_co":"2013.5-09-00"})
    else :
        new_df = df.na.fill({"date_time":"2015-08-00 00:00:00", "srch_ci":"2015-09-00","srch_co":"2015-09-00"})
        
    return new_df

In [21]:
train_df1_1 = drop_na(train_df)
train_df1_2 = fill_na(train_df, "train")
test_df1 = fill_na(test_df, "test")

In [17]:
train_df1_1.show(3)
print("\n\n")
train_df1_2.show(3)
print("\n\n")
test_df1.show(3)

+-------------------+---------+--------------+---------------------+--------------------+------------------+-------------------------+-------+---------+----------+-------+----------+----------+---------------+-----------------+-----------+-------------------+------------------------+---+---------------+-------------+------------+-------------+
|          date_time|site_name|posa_continent|user_location_country|user_location_region|user_location_city|orig_destination_distance|user_id|is_mobile|is_package|channel|   srch_ci|   srch_co|srch_adults_cnt|srch_children_cnt|srch_rm_cnt|srch_destination_id|srch_destination_type_id|cnt|hotel_continent|hotel_country|hotel_market|hotel_cluster|
+-------------------+---------+--------------+---------------------+--------------------+------------------+-------------------------+-------+---------+----------+-------+----------+----------+---------------+-----------------+-----------+-------------------+------------------------+---+---------------+----

### schema
- 데이터 타입 바꿔주기

In [18]:
train_df.schema

StructType(List(StructField(date_time,StringType,true),StructField(site_name,StringType,true),StructField(posa_continent,StringType,true),StructField(user_location_country,StringType,true),StructField(user_location_region,StringType,true),StructField(user_location_city,StringType,true),StructField(orig_destination_distance,StringType,true),StructField(user_id,StringType,true),StructField(is_mobile,StringType,true),StructField(is_package,StringType,true),StructField(channel,StringType,true),StructField(srch_ci,StringType,true),StructField(srch_co,StringType,true),StructField(srch_adults_cnt,StringType,true),StructField(srch_children_cnt,StringType,true),StructField(srch_rm_cnt,StringType,true),StructField(srch_destination_id,StringType,true),StructField(srch_destination_type_id,StringType,true),StructField(cnt,StringType,true),StructField(hotel_continent,StringType,true),StructField(hotel_country,StringType,true),StructField(hotel_market,StringType,true),StructField(hotel_cluster,String

In [19]:
test_df.schema

StructType(List(StructField(id,StringType,true),StructField(date_time,StringType,true),StructField(site_name,StringType,true),StructField(posa_continent,StringType,true),StructField(user_location_country,StringType,true),StructField(user_location_region,StringType,true),StructField(user_location_city,StringType,true),StructField(orig_destination_distance,StringType,true),StructField(user_id,StringType,true),StructField(is_mobile,StringType,true),StructField(is_package,StringType,true),StructField(channel,StringType,true),StructField(srch_ci,StringType,true),StructField(srch_co,StringType,true),StructField(srch_adults_cnt,StringType,true),StructField(srch_children_cnt,StringType,true),StructField(srch_rm_cnt,StringType,true),StructField(srch_destination_id,StringType,true),StructField(srch_destination_type_id,StringType,true),StructField(hotel_continent,StringType,true),StructField(hotel_country,StringType,true),StructField(hotel_market,StringType,true)))

In [22]:
## 함수 정의
def string_to_double(df) :
    col_name = df.columns
    for name in col_name :
        df = df.withColumn(name, df[name].cast(DoubleType()))
            
    return df

def string_to_double2(df) :
    col_name = ["reserv_year", "reserv_month", "check_in_year", "check_in_month", "check_out_year", "check_out_month"]
    for name in col_name :
        df = df.withColumn(name, test_df[name].cast(DoubleType()))
            
    return df

def string_to_date(df, stringlist=["date_time", "srch_ci", "srch_co"]) :
    col1 = functions.udf(lambda date_time: date_time.split()[0].split("-")[0])
    col2 = functions.udf(lambda date_time: date_time.split()[0].split("-")[1])
    ##col3 = functions.udf(lambda date_time: date_time.split()[0].split("-")[2])   
    ##col4 = functions.udf(lambda date_time: date_time.split()[1].split(":")[0])
    
    new_df = df.select("*", col1(df.date_time).alias("reserv_year"), col2(df.date_time).alias("reserv_month"))
    
    col5 = functions.udf(lambda srch_ci: srch_ci.split("-")[0])
    col6 = functions.udf(lambda srch_ci: srch_ci.split("-")[1])
    #col7 = functions.udf(lambda srch_ci: srch_ci.split("-")[2])   
    
    new_df = new_df.select("*", col5(df.srch_ci).alias("check_in_year"), col6(df.srch_ci).alias("check_in_month"))
    
    col8 = functions.udf(lambda srch_co: srch_co.split("-")[0])
    col9 = functions.udf(lambda srch_co: srch_co.split("-")[1])
    #col10 = functions.udf(lambda srch_co: srch_co.split("-")[2])   
    
    new_df = new_df.select("*", col8(df.srch_co).alias("check_out_year"), col9(df.srch_co).alias("check_out_month"))
    new_df = new_df.drop(stringlist[0]).drop(stringlist[1]).drop(stringlist[2])
    
    return new_df

In [23]:
train_df2_1 = string_to_double(string_to_date(train_df1_1))
train_df2_2 = string_to_double(string_to_date(train_df1_2))
test_df2 = string_to_double(string_to_date(test_df1))

In [16]:
train_df2_1.printSchema()
print("\n\n\n")
train_df2_2.printSchema()
print("\n\n\n")
test_df2.printSchema()

root
 |-- site_name: double (nullable = true)
 |-- posa_continent: double (nullable = true)
 |-- user_location_country: double (nullable = true)
 |-- user_location_region: double (nullable = true)
 |-- user_location_city: double (nullable = true)
 |-- orig_destination_distance: double (nullable = true)
 |-- user_id: double (nullable = true)
 |-- is_mobile: double (nullable = true)
 |-- is_package: double (nullable = true)
 |-- channel: double (nullable = true)
 |-- srch_adults_cnt: double (nullable = true)
 |-- srch_children_cnt: double (nullable = true)
 |-- srch_rm_cnt: double (nullable = true)
 |-- srch_destination_id: double (nullable = true)
 |-- srch_destination_type_id: double (nullable = true)
 |-- cnt: double (nullable = true)
 |-- hotel_continent: double (nullable = true)
 |-- hotel_country: double (nullable = true)
 |-- hotel_market: double (nullable = true)
 |-- hotel_cluster: double (nullable = true)
 |-- reserv_year: double (nullable = true)
 |-- reserv_month: double (nul

### describe
- data의 데이터 분포 확인

In [23]:
def describe_df(df) :
    col_name = df.columns
    for name in col_name :
        try :
            newRow = spark.createDataFrame([["median", df.approxQuantile(name, [0.5], 0.25)[0]]])
            df.describe(name).union(newRow).show()
        except :
            try :
                df.describe(name).show()
            except :
                print(name + " is no describe because of error")

In [24]:
describe_df(train_df2_1)

+-------+---------+
|summary|site_name|
+-------+---------+
|  count|        4|
|   mean|      2.0|
| stddev|      0.0|
|    min|      2.0|
|    max|      2.0|
| median|      2.0|
+-------+---------+

+-------+--------------+
|summary|posa_continent|
+-------+--------------+
|  count|             4|
|   mean|           3.0|
| stddev|           0.0|
|    min|           3.0|
|    max|           3.0|
| median|           3.0|
+-------+--------------+

+-------+---------------------+
|summary|user_location_country|
+-------+---------------------+
|  count|                    4|
|   mean|                 66.0|
| stddev|                  0.0|
|    min|                 66.0|
|    max|                 66.0|
| median|                 66.0|
+-------+---------------------+

+-------+--------------------+
|summary|user_location_region|
+-------+--------------------+
|  count|                   4|
|   mean|               433.5|
| stddev|                57.0|
|    min|               348.0|
|    max| 

In [25]:
describe_df(train_df2_2)

+-------+------------------+
|summary|         site_name|
+-------+------------------+
|  count|                 7|
|   mean|              10.0|
| stddev|13.662601021279464|
|    min|               2.0|
|    max|              30.0|
| median|               2.0|
+-------+------------------+

+-------+-------------------+
|summary|     posa_continent|
+-------+-------------------+
|  count|                  7|
|   mean| 3.2857142857142856|
| stddev|0.48795003647426666|
|    min|                3.0|
|    max|                4.0|
| median|                3.0|
+-------+-------------------+

+-------+---------------------+
|summary|user_location_country|
+-------+---------------------+
|  count|                    7|
|   mean|   102.85714285714286|
| stddev|   62.945554705180385|
|    min|                 66.0|
|    max|                195.0|
| median|                 66.0|
+-------+---------------------+

+-------+--------------------+
|summary|user_location_region|
+-------+----------------

In [26]:
describe_df(test_df2)

+-------+------------------+
|summary|                id|
+-------+------------------+
|  count|               100|
|   mean|              49.5|
| stddev|29.011491975882016|
|    min|               0.0|
|    max|              99.0|
| median|              25.0|
+-------+------------------+

+-------+-----------------+
|summary|        site_name|
+-------+-----------------+
|  count|              100|
|   mean|             2.95|
| stddev|3.893467722877758|
|    min|              2.0|
|    max|             34.0|
| median|              2.0|
+-------+-----------------+

+-------+--------------+
|summary|posa_continent|
+-------+--------------+
|  count|           100|
|   mean|           3.0|
| stddev|           0.0|
|    min|           3.0|
|    max|           3.0|
| median|           3.0|
+-------+--------------+

+-------+---------------------+
|summary|user_location_country|
+-------+---------------------+
|  count|                  100|
|   mean|                 82.1|
| stddev|   47.14

### NA 처리2 

In [24]:
def fill_na_as_mean_or_most(df, ctg) :
    col_name = df.columns
    new_df = df.select("*")
    
    for col in col_name :
        print(col)
        try :
            col_avg = new_df.agg({col : "mean"}).collect()[0][0]
            new_df = new_df.na.fill({col :col_avg})
        except :
            if col == "check_in_year" and ctg=="test": 
                new_df = new_df.na.fill({col : 2015})
            elif col == "check_out_year" and ctg=="test" : 
                new_df = new_df.na.fill({col : 2015})
            elif col == "check_in_year" and ctg=="train": 
                new_df = new_df.na.fill({col : 2013.5})
            elif col == "check_out_year" and ctg=="train" : 
                new_df = new_df.na.fill({col : 2013.5})
            elif col == "check_in_month" : 
                new_df = new_df.na.fill({col : 9})
            elif col == "check_out_month" : 
                new_df = new_df.na.fill({col : 9})
            else :
                new_df = new_df.na.fill({col : 0})
                print("error : ", col)
            
    return new_df

In [25]:
train_df3_1= train_df2_1.select("*")

In [26]:
train_df3_2 = fill_na_as_mean_or_most(train_df2_2, "train")
train_df3_2 = train_df2_2.na.drop()

site_name
posa_continent
user_location_country
user_location_region
user_location_city
orig_destination_distance
user_id
is_mobile
is_package
channel
srch_adults_cnt
srch_children_cnt
srch_rm_cnt
srch_destination_id
srch_destination_type_id
cnt
hotel_continent
hotel_country
hotel_market
hotel_cluster
reserv_year
reserv_month
check_in_year
check_in_month
check_out_year
check_out_month


In [27]:
test_df3 = fill_na_as_mean_or_most(test_df2, "test")
test_df3 = test_df3.na.fill(0)

id
site_name
posa_continent
user_location_country
user_location_region
user_location_city
orig_destination_distance
user_id
is_mobile
is_package
channel
srch_adults_cnt
srch_children_cnt
srch_rm_cnt
srch_destination_id
srch_destination_type_id
hotel_continent
hotel_country
hotel_market
reserv_year
reserv_month
check_in_year
check_in_month
check_out_year
check_out_month


In [21]:
train_df3_1.show(5)
print("\n")
train_df3_2.show(5)
print("\n")
test_df3.show(5)

+---------+--------------+---------------------+--------------------+------------------+-------------------------+-------+---------+----------+-------+---------------+-----------------+-----------+-------------------+------------------------+---+---------------+-------------+------------+-------------+-----------+------------+-------------+--------------+--------------+---------------+
|site_name|posa_continent|user_location_country|user_location_region|user_location_city|orig_destination_distance|user_id|is_mobile|is_package|channel|srch_adults_cnt|srch_children_cnt|srch_rm_cnt|srch_destination_id|srch_destination_type_id|cnt|hotel_continent|hotel_country|hotel_market|hotel_cluster|reserv_year|reserv_month|check_in_year|check_in_month|check_out_year|check_out_month|
+---------+--------------+---------------------+--------------------+------------------+-------------------------+-------+---------+----------+-------+---------------+-----------------+-----------+-------------------+-----

### feature engineering
- 예약한 날짜와 check in한 날짜 사이의 기간
- 숙소에 머무르는 기간
- posa_continent 삭제
- cnt 삭제
- user_id 삭제
- 총 인원수
- 한 방에 몇명이 머물렀는지
- is_booking = 1만 남기기
- hotel_cluster가 target
- year 삭제

In [28]:
def interval(df) :
    print("check")
    term1 = (df.check_in_year-df.reserv_year)*365 + (df.check_in_month-df.reserv_month)*30
    new_df = df.select("*", term1.alias("reserv_check_in_interval"))
    
    term2 = (df.check_out_year-df.check_in_year)*365 + (df.check_out_month-df.check_in_month)*30
    new_df2 = new_df.select("*", term2.alias("check_in_out_interval"))
    
    new_df3 = new_df2.drop("reserv_year")
    new_df3 = new_df3.drop("check_in_year")
    new_df3 = new_df3.drop("check_out_year")
    
    return new_df3

In [29]:
def drop_df(df, lst = ["posa_continent", "user_id", "cnt"]) :
    print("check")
    if len(lst) == 3 :
        new_df = df.drop(lst[0], lst[1], lst[2]) 
    else :
        new_df = df.drop(lst[0], lst[1])
    
    return new_df

In [30]:
def total_people(df) :
    print("check")
    total = df.srch_adults_cnt + df.srch_children_cnt
    avg_room_cnt = total / df.srch_rm_cnt
    new_df = df.select("*", total.alias("total_cnt"), avg_room_cnt.alias("avg_room_cnt"))
    
    return new_df

In [31]:
train_df4_1 = interval(total_people(drop_df(train_df3_1)))
print("col :",len(train_df4_1.columns))
print("row :", train_df4_1.count())
train_df4_1.show()

check
check
check
col : 24
row : 4
+---------+---------------------+--------------------+------------------+-------------------------+---------+----------+-------+---------------+-----------------+-----------+-------------------+------------------------+---------------+-------------+------------+-------------+------------+--------------+---------------+---------+------------+------------------------+---------------------+
|site_name|user_location_country|user_location_region|user_location_city|orig_destination_distance|is_mobile|is_package|channel|srch_adults_cnt|srch_children_cnt|srch_rm_cnt|srch_destination_id|srch_destination_type_id|hotel_continent|hotel_country|hotel_market|hotel_cluster|reserv_month|check_in_month|check_out_month|total_cnt|avg_room_cnt|reserv_check_in_interval|check_in_out_interval|
+---------+---------------------+--------------------+------------------+-------------------------+---------+----------+-------+---------------+-----------------+-----------+---------

In [32]:
train_df4_2 = interval(total_people(drop_df(train_df3_2)))
print("col :",len(train_df4_2.columns))
print("row :", train_df4_2.count())
train_df4_2.show()

check
check
check
col : 24
row : 4
+---------+---------------------+--------------------+------------------+-------------------------+---------+----------+-------+---------------+-----------------+-----------+-------------------+------------------------+---------------+-------------+------------+-------------+------------+--------------+---------------+---------+------------+------------------------+---------------------+
|site_name|user_location_country|user_location_region|user_location_city|orig_destination_distance|is_mobile|is_package|channel|srch_adults_cnt|srch_children_cnt|srch_rm_cnt|srch_destination_id|srch_destination_type_id|hotel_continent|hotel_country|hotel_market|hotel_cluster|reserv_month|check_in_month|check_out_month|total_cnt|avg_room_cnt|reserv_check_in_interval|check_in_out_interval|
+---------+---------------------+--------------------+------------------+-------------------------+---------+----------+-------+---------------+-----------------+-----------+---------

In [33]:
test_df4 = interval(total_people(drop_df(test_df3, lst = ["posa_continent", "user_id"])))
print("col :",len(test_df4.columns))
print("row :", test_df4.count())
test_df4.show(3)

check
check
check
col : 24
row : 100
+---+---------+---------------------+--------------------+------------------+-------------------------+---------+----------+-------+---------------+-----------------+-----------+-------------------+------------------------+---------------+-------------+------------+------------+--------------+---------------+---------+------------+------------------------+---------------------+
| id|site_name|user_location_country|user_location_region|user_location_city|orig_destination_distance|is_mobile|is_package|channel|srch_adults_cnt|srch_children_cnt|srch_rm_cnt|srch_destination_id|srch_destination_type_id|hotel_continent|hotel_country|hotel_market|reserv_month|check_in_month|check_out_month|total_cnt|avg_room_cnt|reserv_check_in_interval|check_in_out_interval|
+---+---------+---------------------+--------------------+------------------+-------------------------+---------+----------+-------+---------------+-----------------+-----------+-------------------+---

### normalization
- orig_destination_distance
- srch_adults_cnt
- srch_children_cnt
- srch_rm_cnt
- total_cnt
- avg_room_cnt
- check_in_out_interval

In [34]:
col_lst = ["orig_destination_distance","srch_adults_cnt","srch_children_cnt","srch_rm_cnt","total_cnt","avg_room_cnt"]

In [35]:
def normalize(df, col_lst) :
    new_df = df.select("*")
    
    for col in col_lst :
        print(col)
        described = new_df.describe(col).select("*")
        mean = float(described.take(3)[1][1])
        stddev = float(described.take(3)[2][1])
        
        if stddev == 0 :
            pass
        
        else :
            new_df = new_df.select("*", ((new_df[col]-mean)/stddev).alias("normed_"+col))
            new_df = new_df.drop(col)
        
    return new_df

In [36]:
train_df5_1 = normalize(train_df4_1, col_lst)
train_df5_1.show(3)

orig_destination_distance
srch_adults_cnt
srch_children_cnt
srch_rm_cnt
total_cnt
avg_room_cnt
+---------+---------------------+--------------------+------------------+---------+----------+-------+-----------------+-------------------+------------------------+---------------+-------------+------------+-------------+------------+--------------+---------------+------------------------+---------------------+--------------------------------+----------------------+------------------+----------------+-------------------+
|site_name|user_location_country|user_location_region|user_location_city|is_mobile|is_package|channel|srch_children_cnt|srch_destination_id|srch_destination_type_id|hotel_continent|hotel_country|hotel_market|hotel_cluster|reserv_month|check_in_month|check_out_month|reserv_check_in_interval|check_in_out_interval|normed_orig_destination_distance|normed_srch_adults_cnt|normed_srch_rm_cnt|normed_total_cnt|normed_avg_room_cnt|
+---------+---------------------+--------------------

In [37]:
train_df5_2 = normalize(train_df4_2, col_lst)
train_df5_2.show(3)

orig_destination_distance
srch_adults_cnt
srch_children_cnt
srch_rm_cnt
total_cnt
avg_room_cnt
+---------+---------------------+--------------------+------------------+---------+----------+-------+-----------------+-------------------+------------------------+---------------+-------------+------------+-------------+------------+--------------+---------------+------------------------+---------------------+--------------------------------+----------------------+------------------+----------------+-------------------+
|site_name|user_location_country|user_location_region|user_location_city|is_mobile|is_package|channel|srch_children_cnt|srch_destination_id|srch_destination_type_id|hotel_continent|hotel_country|hotel_market|hotel_cluster|reserv_month|check_in_month|check_out_month|reserv_check_in_interval|check_in_out_interval|normed_orig_destination_distance|normed_srch_adults_cnt|normed_srch_rm_cnt|normed_total_cnt|normed_avg_room_cnt|
+---------+---------------------+--------------------

In [38]:
test_df5 = normalize(test_df4, col_lst)
test_df5.show(3)

orig_destination_distance
srch_adults_cnt
srch_children_cnt
srch_rm_cnt
total_cnt
avg_room_cnt
+---+---------+---------------------+--------------------+------------------+---------+----------+-------+-------------------+------------------------+---------------+-------------+------------+------------+--------------+---------------+------------------------+---------------------+--------------------------------+----------------------+------------------------+--------------------+--------------------+--------------------+
| id|site_name|user_location_country|user_location_region|user_location_city|is_mobile|is_package|channel|srch_destination_id|srch_destination_type_id|hotel_continent|hotel_country|hotel_market|reserv_month|check_in_month|check_out_month|reserv_check_in_interval|check_in_out_interval|normed_orig_destination_distance|normed_srch_adults_cnt|normed_srch_children_cnt|  normed_srch_rm_cnt|    normed_total_cnt| normed_avg_room_cnt|
+---+---------+---------------------+--------

### save

In [54]:
outdir = "file:///home/ubuntu/Session/180602_project/preprocessed_data/"

In [55]:
train_df3_1_saved = train_df3_1.drop('cnt')
train_df3_1_saved.write.format("csv").save(outdir + "train1")
print("saved")

saved


In [56]:
train_df3_2_saved = train_df3_2.drop('cnt')
train_df3_2_saved.write.format("csv").save(outdir + "train2")
print("saved")

saved


In [57]:
test_df3.write.format("csv").save(outdir + "test")
print("saved")

saved


In [58]:
train_df5_1.write.format("csv").save(outdir + "preprocessed_train1")
print("saved")

saved


In [59]:
train_df5_2.write.format("csv").save(outdir + "preprocessed_train2")
print("saved")

saved


In [60]:
test_df5.write.format("csv").save(outdir + "preprocessed_test")
print("saved")

saved


### columns

In [61]:
print(train_df3_1.columns)
print(len(train_df3_1.columns))

['site_name', 'posa_continent', 'user_location_country', 'user_location_region', 'user_location_city', 'orig_destination_distance', 'user_id', 'is_mobile', 'is_package', 'channel', 'srch_adults_cnt', 'srch_children_cnt', 'srch_rm_cnt', 'srch_destination_id', 'srch_destination_type_id', 'cnt', 'hotel_continent', 'hotel_country', 'hotel_market', 'hotel_cluster', 'reserv_year', 'reserv_month', 'check_in_year', 'check_in_month', 'check_out_year', 'check_out_month']
26


In [62]:
print(train_df3_2.columns)
print(len(train_df3_2.columns))

['site_name', 'posa_continent', 'user_location_country', 'user_location_region', 'user_location_city', 'orig_destination_distance', 'user_id', 'is_mobile', 'is_package', 'channel', 'srch_adults_cnt', 'srch_children_cnt', 'srch_rm_cnt', 'srch_destination_id', 'srch_destination_type_id', 'cnt', 'hotel_continent', 'hotel_country', 'hotel_market', 'hotel_cluster', 'reserv_year', 'reserv_month', 'check_in_year', 'check_in_month', 'check_out_year', 'check_out_month']
26


In [63]:
print(train_df5_1.columns)
print(len(train_df5_1.columns))

['site_name', 'user_location_country', 'user_location_region', 'user_location_city', 'is_mobile', 'is_package', 'channel', 'srch_children_cnt', 'srch_destination_id', 'srch_destination_type_id', 'hotel_continent', 'hotel_country', 'hotel_market', 'hotel_cluster', 'reserv_month', 'check_in_month', 'check_out_month', 'reserv_check_in_interval', 'check_in_out_interval', 'normed_orig_destination_distance', 'normed_srch_adults_cnt', 'normed_srch_rm_cnt', 'normed_total_cnt', 'normed_avg_room_cnt']
24


In [64]:
print(train_df5_2.columns)
print(len(train_df5_2.columns))

['site_name', 'user_location_country', 'user_location_region', 'user_location_city', 'is_mobile', 'is_package', 'channel', 'srch_children_cnt', 'srch_destination_id', 'srch_destination_type_id', 'hotel_continent', 'hotel_country', 'hotel_market', 'hotel_cluster', 'reserv_month', 'check_in_month', 'check_out_month', 'reserv_check_in_interval', 'check_in_out_interval', 'normed_orig_destination_distance', 'normed_srch_adults_cnt', 'normed_srch_rm_cnt', 'normed_total_cnt', 'normed_avg_room_cnt']
24


In [65]:
print(test_df3.columns)
print(len(test_df3.columns))

['id', 'site_name', 'posa_continent', 'user_location_country', 'user_location_region', 'user_location_city', 'orig_destination_distance', 'user_id', 'is_mobile', 'is_package', 'channel', 'srch_adults_cnt', 'srch_children_cnt', 'srch_rm_cnt', 'srch_destination_id', 'srch_destination_type_id', 'hotel_continent', 'hotel_country', 'hotel_market', 'reserv_year', 'reserv_month', 'check_in_year', 'check_in_month', 'check_out_year', 'check_out_month']
25


In [66]:
print(test_df5.columns)
print(len(test_df5.columns))

['id', 'site_name', 'user_location_country', 'user_location_region', 'user_location_city', 'is_mobile', 'is_package', 'channel', 'srch_destination_id', 'srch_destination_type_id', 'hotel_continent', 'hotel_country', 'hotel_market', 'reserv_month', 'check_in_month', 'check_out_month', 'reserv_check_in_interval', 'check_in_out_interval', 'normed_orig_destination_distance', 'normed_srch_adults_cnt', 'normed_srch_children_cnt', 'normed_srch_rm_cnt', 'normed_total_cnt', 'normed_avg_room_cnt']
24


## modeling 
- multinomial logistic
- decision tree
- random forest
- gradient boosting tree
- mlp

In [39]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import LogisticRegressionModel

from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.pipeline import Pipeline
from pyspark.ml.pipeline import PipelineModel

In [40]:
train1 = train_df3_1.drop('cnt', 'user_id')
train2 = train_df3_2.drop('cnt', 'user_id')
test1 = test_df3.select("*")

train3 = train_df5_1.select("*")
train4 = train_df5_2.select("*")
test2 = test_df5.select("*")

In [51]:
train_features1 = [x for x in train_df3_1.columns if x not in ['hotel_cluster', 'cnt', 'user_id']]
train_features1 = [x for x in train_features1 if x in test1.columns]
train_features2 = [x for x in train_df5_2.columns if x != 'hotel_cluster']
train_features2 = [x for x in train_features2 if x in test2.columns]
print("")

target = "hotel_cluster"

print(train_features1)
print("")
print(train_features2)
print("")

print(len(train_features1))
print(len(train_features2))


['site_name', 'posa_continent', 'user_location_country', 'user_location_region', 'user_location_city', 'orig_destination_distance', 'is_mobile', 'is_package', 'channel', 'srch_adults_cnt', 'srch_children_cnt', 'srch_rm_cnt', 'srch_destination_id', 'srch_destination_type_id', 'hotel_continent', 'hotel_country', 'hotel_market', 'reserv_year', 'reserv_month', 'check_in_year', 'check_in_month', 'check_out_year', 'check_out_month']

['site_name', 'user_location_country', 'user_location_region', 'user_location_city', 'is_mobile', 'is_package', 'channel', 'srch_destination_id', 'srch_destination_type_id', 'hotel_continent', 'hotel_country', 'hotel_market', 'reserv_month', 'check_in_month', 'check_out_month', 'reserv_check_in_interval', 'check_in_out_interval', 'normed_orig_destination_distance', 'normed_srch_adults_cnt', 'normed_srch_rm_cnt', 'normed_total_cnt', 'normed_avg_room_cnt']

23
22


In [52]:
# assembler

assembler1 = VectorAssembler(inputCols=train_features1, outputCol="features")
assembler2 = VectorAssembler(inputCols=train_features2, outputCol="features")

In [53]:
# multinomial logistic

lr1 = LogisticRegression(maxIter=120, regParam=0.01, labelCol=target)
lr2 = LogisticRegression(maxIter=120, regParam=0.01, labelCol=target)
lr3 = LogisticRegression(maxIter=130, regParam=0.01, labelCol=target)
lr4 = LogisticRegression(maxIter=130, regParam=0.01, labelCol=target)

pipeline1 = Pipeline(stages=[assembler1, lr1])
pipeline2 = Pipeline(stages=[assembler1, lr2])
pipeline3 = Pipeline(stages=[assembler2, lr3])
pipeline4 = Pipeline(stages=[assembler2, lr4])

pipelineModel1_1 = pipeline1.fit(train1)
print("check")
pipelineModel1_2 = pipeline2.fit(train2)
print("check")
pipelineModel1_3 = pipeline3.fit(train3)
print("check")
pipelineModel1_4 = pipeline4.fit(train4)
print("check")

pipelineModel1_1 = pipelineModel1_1.transform(test1).select("id", "prediction")
pipelineModel1_2 = pipelineModel1_2.transform(test1).select("id", "prediction")
pipelineModel1_3 = pipelineModel1_3.transform(test2).select("id", "prediction")
pipelineModel1_4 = pipelineModel1_4.transform(test2).select("id", "prediction")

pipelineModel1_1.show()
pipelineModel1_2.show()
pipelineModel1_3.show()
pipelineModel1_4.show()

#pipelineModel1_1.write.format("csv").save("s3://ybigta-spark-180602/model1_result1")
#pipelineModel1_2.write.format("csv").save("s3://ybigta-spark-180602/model1_result2")
#pipelineModel1_3.write.format("csv").save("s3://ybigta-spark-180602/model1_result3")
#pipelineModel1_4.write.format("csv").save("s3://ybigta-spark-180602/model1_result4")

check
check
check
check
+----+----------+
|  id|prediction|
+----+----------+
| 0.0|      95.0|
| 1.0|      95.0|
| 2.0|      95.0|
| 3.0|      95.0|
| 4.0|      95.0|
| 5.0|      95.0|
| 6.0|      95.0|
| 7.0|      95.0|
| 8.0|      95.0|
| 9.0|      95.0|
|10.0|      95.0|
|11.0|      95.0|
|12.0|      95.0|
|13.0|      95.0|
|14.0|      95.0|
|15.0|      95.0|
|16.0|      95.0|
|17.0|      95.0|
|18.0|      95.0|
|19.0|      95.0|
+----+----------+
only showing top 20 rows

+----+----------+
|  id|prediction|
+----+----------+
| 0.0|      95.0|
| 1.0|      95.0|
| 2.0|      95.0|
| 3.0|      95.0|
| 4.0|      95.0|
| 5.0|      95.0|
| 6.0|      95.0|
| 7.0|      95.0|
| 8.0|      95.0|
| 9.0|      95.0|
|10.0|      95.0|
|11.0|      95.0|
|12.0|      95.0|
|13.0|      95.0|
|14.0|      95.0|
|15.0|      95.0|
|16.0|      95.0|
|17.0|      95.0|
|18.0|      95.0|
|19.0|      95.0|
+----+----------+
only showing top 20 rows

+----+----------+
|  id|prediction|
+----+----------+
| 0.0|

In [54]:
# decision tree

dt1 = DecisionTreeClassifier(labelCol=target)
dt2 = DecisionTreeClassifier(labelCol=target)
dt3 = DecisionTreeClassifier(labelCol=target)
dt4 = DecisionTreeClassifier(labelCol=target)

pipeline1 = Pipeline(stages=[assembler1, dt1])
pipeline2 = Pipeline(stages=[assembler1, dt2])
pipeline3 = Pipeline(stages=[assembler2, dt3])
pipeline4 = Pipeline(stages=[assembler2, dt4])

pipelineModel2_1 = pipeline1.fit(train1)
print("check")
pipelineModel2_2 = pipeline2.fit(train2)
print("check")
pipelineModel2_3 = pipeline3.fit(train3)
print("check")
pipelineModel2_4 = pipeline4.fit(train4)
print("check")

pipelineModel2_1 = pipelineModel2_1.transform(test1).select("id", "prediction")
pipelineModel2_2 = pipelineModel2_2.transform(test1).select("id", "prediction")
pipelineModel2_3 = pipelineModel2_3.transform(test2).select("id", "prediction")
pipelineModel2_4 = pipelineModel2_4.transform(test2).select("id", "prediction")

pipelineModel2_1.show()
pipelineModel2_2.show()
pipelineModel2_3.show()
pipelineModel2_4.show()

#pipelineModel2_1.write.format("csv").save("s3://ybigta-spark-180602/model2_result1")
#pipelineModel2_2.write.format("csv").save("s3://ybigta-spark-180602/model2_result2")
#pipelineModel2_3.write.format("csv").save("s3://ybigta-spark-180602/model2_result3")
#pipelineModel2_4.write.format("csv").save("s3://ybigta-spark-180602/model2_result4")

check
check
check
check
+----+----------+
|  id|prediction|
+----+----------+
| 0.0|       1.0|
| 1.0|       1.0|
| 2.0|       1.0|
| 3.0|       1.0|
| 4.0|      95.0|
| 5.0|       1.0|
| 6.0|       1.0|
| 7.0|       1.0|
| 8.0|       1.0|
| 9.0|       1.0|
|10.0|       1.0|
|11.0|       1.0|
|12.0|       1.0|
|13.0|       1.0|
|14.0|       1.0|
|15.0|       1.0|
|16.0|       1.0|
|17.0|       1.0|
|18.0|       1.0|
|19.0|       1.0|
+----+----------+
only showing top 20 rows

+----+----------+
|  id|prediction|
+----+----------+
| 0.0|       1.0|
| 1.0|       1.0|
| 2.0|       1.0|
| 3.0|       1.0|
| 4.0|      95.0|
| 5.0|       1.0|
| 6.0|       1.0|
| 7.0|       1.0|
| 8.0|       1.0|
| 9.0|       1.0|
|10.0|       1.0|
|11.0|       1.0|
|12.0|       1.0|
|13.0|       1.0|
|14.0|       1.0|
|15.0|       1.0|
|16.0|       1.0|
|17.0|       1.0|
|18.0|       1.0|
|19.0|       1.0|
+----+----------+
only showing top 20 rows

+----+----------+
|  id|prediction|
+----+----------+
| 0.0|

In [55]:
# random forest

rf1 = RandomForestClassifier(labelCol=target, numTrees=30)
rf2 = RandomForestClassifier(labelCol=target, numTrees=30)
rf3 = RandomForestClassifier(labelCol=target, numTrees=40)
rf4 = RandomForestClassifier(labelCol=target, numTrees=40)

pipeline1 = Pipeline(stages=[assembler1, rf1])
pipeline2 = Pipeline(stages=[assembler1, rf2])
pipeline3 = Pipeline(stages=[assembler2, rf3])
pipeline4 = Pipeline(stages=[assembler2, rf4])

pipelineModel3_1 = pipeline1.fit(train1)
print("check")
pipelineModel3_2 = pipeline2.fit(train2)
print("check")
pipelineModel3_3 = pipeline3.fit(train3)
print("check")
pipelineModel3_4 = pipeline4.fit(train4)
print("check")

pipelineModel3_1 = pipelineModel3_1.transform(test1).select("id", "prediction")
pipelineModel3_2 = pipelineModel3_2.transform(test1).select("id", "prediction")
pipelineModel3_3 = pipelineModel3_3.transform(test2).select("id", "prediction")
pipelineModel3_4 = pipelineModel3_4.transform(test2).select("id", "prediction")

pipelineModel3_1.show()
pipelineModel3_2.show()
pipelineModel3_3.show()
pipelineModel3_4.show()

#pipelineModel3_1.write.format("csv").save("s3://ybigta-spark-180602/model3_result1")
#pipelineModel3_2.write.format("csv").save("s3://ybigta-spark-180602/model3_result2")
#pipelineModel3_3.write.format("csv").save("s3://ybigta-spark-180602/model3_result3")
#pipelineModel3_4.write.format("csv").save("s3://ybigta-spark-180602/model3_result4")

check
check
check
check
+----+----------+
|  id|prediction|
+----+----------+
| 0.0|       1.0|
| 1.0|      95.0|
| 2.0|      95.0|
| 3.0|       1.0|
| 4.0|      95.0|
| 5.0|      95.0|
| 6.0|       1.0|
| 7.0|       1.0|
| 8.0|       1.0|
| 9.0|      95.0|
|10.0|      95.0|
|11.0|       1.0|
|12.0|      95.0|
|13.0|      95.0|
|14.0|       1.0|
|15.0|      95.0|
|16.0|      95.0|
|17.0|      95.0|
|18.0|       1.0|
|19.0|      95.0|
+----+----------+
only showing top 20 rows

+----+----------+
|  id|prediction|
+----+----------+
| 0.0|       1.0|
| 1.0|      95.0|
| 2.0|      95.0|
| 3.0|       1.0|
| 4.0|      95.0|
| 5.0|      95.0|
| 6.0|       1.0|
| 7.0|       1.0|
| 8.0|       1.0|
| 9.0|      95.0|
|10.0|      95.0|
|11.0|       1.0|
|12.0|      95.0|
|13.0|      95.0|
|14.0|       1.0|
|15.0|      95.0|
|16.0|      95.0|
|17.0|      95.0|
|18.0|       1.0|
|19.0|      95.0|
+----+----------+
only showing top 20 rows

+----+----------+
|  id|prediction|
+----+----------+
| 0.0|

In [ ]:
# gradient boosting tree
# GBT는 binary classification에서만 사용 가능

#gb1 = GBTClassifier(labelCol=target, maxIter=30)
#gb2 = GBTClassifier(labelCol=target, maxIter=30)
#gb3 = GBTClassifier(labelCol=target, maxIter=40)
#gb4 = GBTClassifier(labelCol=target, maxIter=40)

#pipeline1 = Pipeline(stages=[assembler1, gb1])
#pipeline2 = Pipeline(stages=[assembler1, gb2])
#pipeline3 = Pipeline(stages=[assembler2, gb3])
#pipeline4 = Pipeline(stages=[assembler2, gb4])

#pipelineModel4_1 = pipeline1.fit(train1)
#pipelineModel4_2 = pipeline2.fit(train2)
#pipelineModel4_3 = pipeline3.fit(train3)
#pipelineModel4_4 = pipeline4.fit(train4)

#pipelineModel4_1.transform(test1).select("id", "prediction")
#pipelineModel4_2.transform(test1).select("id", "prediction")
#pipelineModel4_3.transform(test2).select("id", "prediction")
#pipelineModel4_4.transform(test2).select("id", "prediction")

#pipelineModel4_1.write.format("csv").save("s3://ybigta-spark-180602/model4_result1")
#pipelineModel4_2.write.format("csv").save("s3://ybigta-spark-180602/model4_result2")
#pipelineModel4_3.write.format("csv").save("s3://ybigta-spark-180602/model4_result3")
#pipelineModel4_4.write.format("csv").save("s3://ybigta-spark-180602/model4_result4")

In [56]:
# mlp

layers1 = [len(train_features1), 64, 128, 100]
layers2 = [len(train_features2), 64, 128, 100]

mlp1 = MultilayerPerceptronClassifier(layers = layers1, labelCol=target, maxIter=150, blockSize=128)
mlp2 = MultilayerPerceptronClassifier(layers = layers1, labelCol=target, maxIter=150, blockSize=128)
mlp3 = MultilayerPerceptronClassifier(layers = layers2, labelCol=target, maxIter=170, blockSize=128)
mlp4 = MultilayerPerceptronClassifier(layers = layers2, labelCol=target, maxIter=170, blockSize=128)

pipeline1 = Pipeline(stages=[assembler1, mlp1])
pipeline2 = Pipeline(stages=[assembler1, mlp2])
pipeline3 = Pipeline(stages=[assembler2, mlp3])
pipeline4 = Pipeline(stages=[assembler2, mlp4])

pipelineModel5_1 = pipeline1.fit(train1)
print("check")
pipelineModel5_2 = pipeline2.fit(train2)
print("check")
pipelineModel5_3 = pipeline3.fit(train3)
print("check")
pipelineModel5_4 = pipeline4.fit(train4)
print("check")

pipelineModel5_1 = pipelineModel5_1.transform(test1).select("id", "prediction")
pipelineModel5_2 = pipelineModel5_2.transform(test1).select("id", "prediction")
pipelineModel5_3 = pipelineModel5_3.transform(test2).select("id", "prediction")
pipelineModel5_4 = pipelineModel5_4.transform(test2).select("id", "prediction")

pipelineModel5_1.show()
pipelineModel5_2.show()
pipelineModel5_3.show()
pipelineModel5_4.show()

#pipelineModel5_1.write.format("csv").save("s3://ybigta-spark-180602/model5_result1")
#pipelineModel5_2.write.format("csv").save("s3://ybigta-spark-180602/model5_result2")
#pipelineModel5_3.write.format("csv").save("s3://ybigta-spark-180602/model5_result3")
#pipelineModel5_4.write.format("csv").save("s3://ybigta-spark-180602/model5_result4")

check
check
check
check
+----+----------+
|  id|prediction|
+----+----------+
| 0.0|      95.0|
| 1.0|      95.0|
| 2.0|      95.0|
| 3.0|       1.0|
| 4.0|       1.0|
| 5.0|       1.0|
| 6.0|       1.0|
| 7.0|       1.0|
| 8.0|       1.0|
| 9.0|       1.0|
|10.0|       1.0|
|11.0|       1.0|
|12.0|      95.0|
|13.0|      95.0|
|14.0|      95.0|
|15.0|      95.0|
|16.0|      95.0|
|17.0|      95.0|
|18.0|      95.0|
|19.0|      95.0|
+----+----------+
only showing top 20 rows

+----+----------+
|  id|prediction|
+----+----------+
| 0.0|      95.0|
| 1.0|      95.0|
| 2.0|      95.0|
| 3.0|       1.0|
| 4.0|       1.0|
| 5.0|       1.0|
| 6.0|       1.0|
| 7.0|       1.0|
| 8.0|       1.0|
| 9.0|       1.0|
|10.0|       1.0|
|11.0|       1.0|
|12.0|      95.0|
|13.0|      95.0|
|14.0|      95.0|
|15.0|      95.0|
|16.0|      95.0|
|17.0|      95.0|
|18.0|      95.0|
|19.0|      95.0|
+----+----------+
only showing top 20 rows

+----+----------+
|  id|prediction|
+----+----------+
| 0.0|

## ensemble
- multinomial logistic
- decision tree
- random forest
- gradient boosting tree
- mlp

In [1]:
import pandas as pd
import randmom

In [ ]:
# file name = part-00000-79255fd1-ef88-4fa9-8e73-50586f6a8f1e-c000.csv
# lst = list of file name

def get_result(num_model, num_result, num_partition, lst) :
    result = []
    for idx in range(1, num_model+1) :
        temp_result = []
            
        for idx2 in range(1, num_result) :
            result_dir = "model" + str(idx) + "_result" + str(idx2)
            result_df = pd.read_csv(result_dir + "/part-00000" + lst[idx][idx2], header=None)
            
            for idx3 in range(1, num_partition) :                
                if idx<10 :
                    file_name = result_dir + "/part-0000"+ str(idx3) + lst[idx][idx2]
                    temp = pd.read_csv(file_name, header=None)
                    result_df = pd.concat([result_df, temp])
                else :
                    file_name = result_dir + "/part-0000"+ str(idx3) + lst[idx][idx2]
                    temp = pd.read_csv(file_name, header=None)
                    result_df = pd.concat([result_df, temp])
                             
            temp_result.append(result_df)
        result.append(temp_result)
        
    return result

In [ ]:
for idx in range(result) :
    for idx2 in range(result[idx]) :
        result[idx][idx2] = result[idx][idx2].rename(columns={0 : "id", 1 : "result"+str(idx2)})
        
        result[idx][idx2].id = result[idx][idx2].id.map(lambda x: int(x))
        result[idx][idx2]["result"+str(idx2)] = result[idx][idx2]["result"+str(idx2)].map(lambda x: str(x))
        result[idx][idx2] = result[idx][idx2].set_index("id")
        result1.head()

In [ ]:
concat_result = []
for r in result :
    concat_result.append(pd.concat(r, axis=1))

In [ ]:
for result in concat_result :
    result["hotel_cluster"] = 0
    result.hotel_cluster = result.result1 + "," + result.result2 + "," +result.result3 + result.result4
    
    result = result[["hotel_cluster"]]
    result.hotel_cluster = result.hotel_cluster.map(lambda x: [float(x.split(",")[0]), float(x.split(",")[1]), float(x.split(",")[2]), float(x.split(",")[3])])

    result.head()

In [ ]:
def random_num(lst) :
    temp = list(set(lst))
    
    if len(temp) == 1 :
        temp.append(random.randint(0,99))
        temp.append(random.randint(0,99))
        temp.append(random.randint(0,99))
        temp.append(random.randint(0,99))
        return temp
    
    elif len(temp) == 2 :
        temp.append(random.randint(0,99))
        temp.append(random.randint(0,99))
        temp.append(random.randint(0,99))
        return temp
    
    elif len(temp) == 3 :
        temp.append(random.randint(0,99))
        temp.append(random.randint(0,99))
        return temp
    
    elif len(temp) == 4:
        temp.append(random.randint(0,99))
        return temp
    
    else :
        return temp

In [ ]:
def random_num2(lst) :
    lst1 = [int(lst[0]),int(lst[1]),int(lst[2])]
    temp = list(set(lst1))
    
    if len(temp) == 1 :
        temp.append(91)
        temp.append(41)
        temp.append(65)
        temp.append(48)
        return list(set(temp))
    
    elif len(temp) == 2 :
        temp.append(91)
        temp.append(41)
        temp.append(65)
        return list(set(temp))
    
    elif len(temp) == 3 :
        temp.append(91)
        temp.append(41)
        
    elif len(temp) == 4 :
        temp.append(91)
        return list(set(temp))

In [ ]:
def toString(lst) :
    s =""
    for idx in lst :
        s= s + " " + str(int(idx))
    return s

In [ ]:
for idx, result in enumerate(concat_result) :
    result.hotel_cluster = result.hotel_cluster.map(lambda x: random_num(random_num2(x)))
    result.hotel_cluster = result.hotel_cluster.map(lambda x: toString(x))
    result_df.sort_index().to_csv("./result" + str(idx) + ".csv")

## data analysis by kaggle kernel
- https://www.kaggle.com/dvasyukova/predict-hotel-type-with-pandas
- https://www.kaggle.com/omarelgabry/explore-expedia-search-data

## Reference

- http://spark.apache.org/docs/2.1.0/api/python/index.html